# Module 4.2: Zero Noise Extrapolation (ZNE)

**The Concept:** Real quantum computers are noisy. As we increase the depth of a circuit, the signal decays into noise (entropy).

**The Solution:** Instead of trying to *remove* noise, we **amplify** it intentionally.
1. Measure at Noise Level 1x (Base)
2. Measure at Noise Level 3x (Stretched Pulses)
3. Measure at Noise Level 5x

By fitting a curve to these points, we can extrapolate backwards to **Noise Level 0**.

**The Innovation:** Standard ZNE uses a linear fit. Here, we implement a **Hyperbolic Tangent (Tanh) Decay Model**, which better captures the physics of decoherence in large systems.

## 🧠 Hardware Access
xperiment require an IBM Quantum API key.  
Get one here: [quantum.ibm.com](https://quantum.ibm.com)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
from qiskit.circuit.library import EfficientSU2, ZZFeatureMap
from qiskit.quantum_info import SparsePauliOp
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit_ibm_runtime import QiskitRuntimeService, EstimatorV2 as Estimator, EstimatorOptions

# --- CONFIGURATION ---
# REPLACE WITH YOUR IBM QUANTUM API KEY
API_KEY = "YOUR_IBM_QUANTUM_API_KEY_HERE"
BACKEND_NAME = "ibm_torino" # Or 'ibm_kyoto', 'ibm_osaka'
NUM_QUBITS = 50
SHOTS = 4096
NOISE_FACTORS = [1, 3, 5]  # We measure at these noise levels

print(f"Targeting Backend: {BACKEND_NAME} with {NUM_QUBITS} Qubits")

In [ ]:
# --- DEFINING THE CUSTOM MODEL ---
# Logic: Signal decays as a tanh function of noise level (x)
# f(x) = a + b * tanh(c * x)
# The Zero Noise Limit (x=0) is 'a'.

def tanh_decay_model(x, a, b, c):
    return a + b * np.tanh(c * x)

In [ ]:
# --- 1. BUILD & TRANSPILE CIRCUIT ---

service = QiskitRuntimeService(channel="ibm_quantum_platform", token=API_KEY)
backend = service.backend(BACKEND_NAME)

# Create a Feature Map + Ansatz (Standard ML Pattern)
print("Constructing Circuit...")
feature_map = ZZFeatureMap(feature_dimension=NUM_QUBITS, reps=1, entanglement='linear')
ansatz = EfficientSU2(num_qubits=NUM_QUBITS, reps=1, entanglement='linear')
circuit = feature_map.compose(ansatz)

print("Transpiling for Hardware...")
pm = generate_preset_pass_manager(backend=backend, optimization_level=3)
isa_circuit = pm.run(circuit)

# Define Observable (Measure Z on Qubit 0)
observable = SparsePauliOp(["I" * (NUM_QUBITS - 1) + "Z"])
isa_observable = observable.apply_layout(isa_circuit.layout)

# Generate Random Parameters for the experiment
np.random.seed(42)
full_params = np.random.uniform(0, 2*np.pi, circuit.num_parameters)

In [ ]:
# --- 2. EXECUTE WITH ZNE ---

print(f"--- SUBMITTING JOB (Noise Factors: {NOISE_FACTORS}) ---")

options = EstimatorOptions()
options.default_shots = SHOTS
options.resilience_level = 2
options.resilience.zne_mitigation = True
options.resilience.zne.noise_factors = NOISE_FACTORS
# We set a dummy extrapolator because we will do the math ourselves
options.resilience.zne.extrapolator = "linear"

estimator = Estimator(mode=backend, options=options)

# Run
job = estimator.run([(isa_circuit, [isa_observable], [full_params])])
print(f"Job ID: {job.job_id()}")
print("Waiting for completion (This may take minutes to hours depending on queue)...")

result = job.result()

In [ ]:
# --- 3. ANALYSIS & EXTRAPOLATION ---

# Extract Raw Data
# Qiskit V2 returns expectation values at each noise factor in 'evs_noise_factors'
raw_evs = result[0].data.evs_noise_factors[0].flatten()

print("\n--- RAW DATA FROM QUANTUM COMPUTER ---")
for nf, ev in zip(NOISE_FACTORS, raw_evs):
    print(f"Noise Factor {nf:.1f}x : Expectation = {ev:.5f}")

print("\n--- FITTING CUSTOM TANH MODEL ---")

# Fit the data to: a + b * tanh(c * x)
# Initial guess (p0) helps convergence
try:
    popt, pcov = curve_fit(tanh_decay_model, NOISE_FACTORS, raw_evs, p0=[-0.2, 0.1, 0.5], maxfev=5000)

    # The Zero Noise Limit is f(0) = a
    zne_limit_tanh = popt[0]

    print(f"Model Parameters: a={popt[0]:.4f}, b={popt[1]:.4f}, c={popt[2]:.4f}")
    print(f">> Extrapolated Value (x=0): {zne_limit_tanh:.5f}")

    # Compare with Standard Linear Fit
    slope, intercept = np.polyfit(NOISE_FACTORS, raw_evs, 1)
    zne_limit_linear = intercept
    print(f">> Standard Linear Value     : {zne_limit_linear:.5f}")
    
except Exception as e:
    print(f"Fitting failed: {e}")
    zne_limit_tanh = 0
    zne_limit_linear = 0
    popt = [0, 0, 0]
    slope, intercept = 0, 0

In [ ]:
# --- 4. VISUALIZATION ---

plt.figure(figsize=(10, 6))

# Plot Measured Points
plt.scatter(NOISE_FACTORS, raw_evs, color='red', label='Measured Data (IBM Heron)', zorder=5, s=100)

# Plot Fits
x_range = np.linspace(0, 6, 100)

# Tanh Curve
plt.plot(x_range, tanh_decay_model(x_range, *popt), 
         label=f'Custom Tanh Fit (ZNE={zne_limit_tanh:.3f})', color='blue', linewidth=2)

# Linear Line
plt.plot(x_range, slope*x_range + intercept, 
         label=f'Standard Linear Fit (ZNE={zne_limit_linear:.3f})', color='gray', linestyle='--', linewidth=2)

# Decoration
plt.scatter([0], [zne_limit_tanh], color='blue', marker='*', s=300, label='Projected Zero Noise')
plt.axvline(x=0, color='black', linewidth=1)
plt.xlabel('Noise Amplification Factor (Scaling)')
plt.ylabel('Expectation Value <Z>')
plt.title('Custom ZNE: Tanh vs Linear Extrapolation (50 Qubits)', fontsize=14)
plt.legend()
plt.grid(True, alpha=0.3)

plt.show()